In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# set target url (disclosure search page) as driver 
driver = webdriver.Chrome()
driver.get('https://kind.krx.co.kr/disclosure/searchdisclosurebycorp.do?method=searchDisclosureByCorpMain')
original_window = driver.current_window_handle

# type target company in search bar 
company_element = driver.find_element(By.ID, 'AKCKwd')
company_element.send_keys('KT&G')

# click on timeframe (6 months)
date_element = driver.find_element(By.CLASS_NAME, 'ord-03')
date_element.click()

# type target document title (Disclosure of Corporate Governance Report)
report_element = driver.find_element(By.ID, 'reportNmTemp')
report_element.send_keys('기업지배구조 보고서 공시')

# click on search button
search_element = driver.find_element(By.CSS_SELECTOR, 'a.btn-sprite.search-btn')
search_element.click()

# click on report link 
link_element = driver.find_element(By.LINK_TEXT, '기업지배구조 보고서 공시') 
link_element.click()

In [ ]:
# switch driver to the opened report link window

all_window_handles = driver.window_handles
for handle in all_window_handles:
    if handle != original_window:
        driver.switch_to.window(handle)
        break
print(f'New window: {driver.title}')

New window: [케이티앤지] 기업지배구조 보고서 공시


In [ ]:
# parse through page and extract the shareholder voting results table 

try:
    # wait for iframe (isolated HTML doc embedded inside the report's main page) to load then switch to it 
    wait = WebDriverWait(driver, 10)
    iframe = wait.until(
        EC.presence_of_element_located((By.ID, "docViewFrm"))
    )
    driver.switch_to.frame(iframe)

    # wait for target table to load in, then locate 
    css_selector = 'table-group[aclass="krx-cg_VotingResultsOfTheGeneralMeetingOfShareholdersAbstract"] table.fact-table'   
    fact_table_element = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))
    )

    # set up for beautiful soup parsing 
    # convert selenium web element to string
    table_html_string = fact_table_element.get_attribute('outerHTML')
    soup = BeautifulSoup(table_html_string, 'html.parser')
    
    # extract headers from all <th> tags
    scraped_headers = [th.get_text(strip=True) for th in soup.find_all('th')]
    
    # use pandas to read all table elements 
    dfs = pd.read_html(table_html_string, header=None)

    if dfs:
        df = dfs[0]

        # rename with headers from both axes 
        final_headers = ['총회', '의안'] + scraped_headers[1:]
        
        # clean the df to match the number of headers 
        df = df.iloc[1:]
        df.reset_index(drop=True, inplace=True)
        
        # match column names 
        if len(final_headers) == len(df.columns):
            df.columns = final_headers
        else:
            print("Error: The number of columns does not match the number of headers.")
            print(f"Number of DataFrame columns: {len(df.columns)}")
            print(f"Number of headers: {len(final_headers)}")
            print(f"Scraped headers: {scraped_headers}")
    else:
        print("No tables found in the HTML.")

except Exception as e:
    print(f"Error: {e}")

finally:
    # switch back to main 
    driver.switch_to.default_content()

C:\Users\Owner\AppData\Local\Temp\ipykernel_21088\829292074.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(table_html_string, header=None)


In [139]:
df

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
0,제38기 정기 주주총회,제2-1호 의안,특별(Extraordinary),이사의 인원수 명확화,가결(Approved),107856043,91736706,91628348,99.9,108358.0,0.1
1,제38기 정기 주주총회,제2-2호 의안,특별(Extraordinary),감사위원 선임 관련 조문 정비,가결(Approved),107856043,91736706,77909093,84.9,13827613.0,15.1
2,제38기 정기 주주총회,제2-3호 의안,특별(Extraordinary),대표이사 사장 선임 방법 명확화,가결(Approved),107856043,74689281,53946867,72.2,20742414.0,27.8
3,제38기 정기 주주총회,제2-4호 의안,특별(Extraordinary),분기배당기준일 변경,가결(Approved),107856043,91736706,91626520,99.9,110186.0,0.1
4,제38기 정기 주주총회,제3호 의안,보통(Ordinary),사내이사 이상학 선임의 건,가결(Approved),107856043,91736706,90539809,98.7,1196897.0,1.3
5,제38기 정기 주주총회,제4-1호 의안,보통(Ordinary),사외이사 손관수 선임의 건,가결(Approved),107856043,91736706,89269740,97.3,2466966.0,2.7
6,제38기 정기 주주총회,제4-2호 의안,보통(Ordinary),사외이사 이지희 선임의 건,가결(Approved),107856043,91736706,90424942,98.6,1311764.0,1.4
7,제38기 정기 주주총회,제5호 의안,보통(Ordinary),감사위원회 위원 손관수 선임의 건,가결(Approved),107856043,74689281,73254834,98.1,1434447.0,1.9
8,제38기 정기 주주총회,제6호 의안,보통(Ordinary),이사 보수한도 승인의 건,가결(Approved),107856043,87807386,87469388,99.6,337998.0,0.4
9,제37기 정기 주주총회,제1호 의안,보통(Ordinary),제37기 재무제표 및 이익잉여금처분계산서 승인의 건,가결(Approved),112809923,87368552,83585189,95.7,3783363.0,4.3


In [141]:
ranked_voting = df[df['반대 기권 등 주식수'].isna()]
majority_voting = df[df['반대 기권 등 주식수'].notna()]

In [143]:
ranked_voting

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
16,제37기 정기 주주총회,제3-1호 의안,보통(Ordinary),대표이사 사장 방경만 선임의 건 (KT&G 이사회 안),가결(Approved),112809923,165207264,84097688,50.9,NaN,0.0
17,제37기 정기 주주총회,제3-2호 의안,보통(Ordinary),사외이사 임민규 선임의 건 (KT&G 이사회 안),부결(Not approved),112809923,165207264,24505618,14.8,NaN,0.0
18,제37기 정기 주주총회,제3-3호 의안,보통(Ordinary),사외이사 손동환 선임의 건 (주주제안_중소기업은행),가결(Approved),112809923,165207264,56603958,34.3,NaN,0.0


In [142]:
majority_voting

,총회,의안,결의 구분,회의 목적사항,가결 여부,의결권 있는 발행주식 총수(1),(1) 중 의결권 행사 주식수,찬성주식수,찬성 주식 비율 (%),반대 기권 등 주식수,반대 기권 등 주식 비율 (%)
0,제38기 정기 주주총회,제2-1호 의안,특별(Extraordinary),이사의 인원수 명확화,가결(Approved),107856043,91736706,91628348,99.9,108358.0,0.1
1,제38기 정기 주주총회,제2-2호 의안,특별(Extraordinary),감사위원 선임 관련 조문 정비,가결(Approved),107856043,91736706,77909093,84.9,13827613.0,15.1
2,제38기 정기 주주총회,제2-3호 의안,특별(Extraordinary),대표이사 사장 선임 방법 명확화,가결(Approved),107856043,74689281,53946867,72.2,20742414.0,27.8
3,제38기 정기 주주총회,제2-4호 의안,특별(Extraordinary),분기배당기준일 변경,가결(Approved),107856043,91736706,91626520,99.9,110186.0,0.1
4,제38기 정기 주주총회,제3호 의안,보통(Ordinary),사내이사 이상학 선임의 건,가결(Approved),107856043,91736706,90539809,98.7,1196897.0,1.3
5,제38기 정기 주주총회,제4-1호 의안,보통(Ordinary),사외이사 손관수 선임의 건,가결(Approved),107856043,91736706,89269740,97.3,2466966.0,2.7
6,제38기 정기 주주총회,제4-2호 의안,보통(Ordinary),사외이사 이지희 선임의 건,가결(Approved),107856043,91736706,90424942,98.6,1311764.0,1.4
7,제38기 정기 주주총회,제5호 의안,보통(Ordinary),감사위원회 위원 손관수 선임의 건,가결(Approved),107856043,74689281,73254834,98.1,1434447.0,1.9
8,제38기 정기 주주총회,제6호 의안,보통(Ordinary),이사 보수한도 승인의 건,가결(Approved),107856043,87807386,87469388,99.6,337998.0,0.4
9,제37기 정기 주주총회,제1호 의안,보통(Ordinary),제37기 재무제표 및 이익잉여금처분계산서 승인의 건,가결(Approved),112809923,87368552,83585189,95.7,3783363.0,4.3


In [ ]:
driver.quit()